In [1]:
import pandas as pd
from scipy import stats as st

In [2]:
data = pd.read_csv('puchkov.k@phystech.edu.csv', header=None)

In [3]:
data.sample(5)

,0,1,2,3,4,5,6
23,-0.917317,0.532543,-1.519402,-0.090229,-1.357987,0.384302,1.281841
29,-2.433938,0.484132,-1.477338,0.440098,-1.139790,1.673886,1.634275
11,-2.035249,2.988589,0.789031,0.960914,0.589665,0.058482,-1.003099
10,-0.392056,1.490833,1.468181,0.117432,0.586793,-1.191817,1.727964
4,-2.002677,-0.728479,0.052631,-0.329110,0.567376,2.192174,0.819236


In [4]:
data.describe()

,0,1,2,3,4,5,6
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,-0.972094,0.848470,0.100682,0.498381,0.247933,0.580504,1.124463
std,0.931788,1.109865,1.158746,0.948012,0.846485,0.997153,1.177982
min,-2.579535,-1.664701,-1.859575,-0.781099,-1.357987,-2.172264,-1.130958
25%,-1.766953,0.212501,-0.814459,-0.182381,-0.213341,0.114758,0.648069
50%,-0.918178,1.066367,0.228887,0.474213,0.384913,0.638803,1.267508
75%,-0.415852,1.503958,0.832125,0.944106,0.669977,1.309557,1.925814
max,0.610077,2.988589,2.318492,3.796164,1.912020,2.411502,3.861457


In [8]:
data.shape

(30, 7)

Воспользуемся t-критерем о сравнении средних для всех пар из столбцов выборки. Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего?

In [6]:
from itertools import combinations

def pairwise_ttest(data: pd.DataFrame, alpha: float = 0.05) -> int:
    """Returns count of pairwise columns with different means and its pvals."""
    cols = 0
    pvals = []

    for col1, col2 in combinations(data.columns, 2):
        # p-value для критерия Стьюдента для независимых выборок
        pvalue = st.ttest_ind(data[col1], data[col2])[1]
        pvals.append(pvalue)
        if pvalue <= 0.05:
            cols += 1
    return cols, pvals

print(f'Для {pairwise_ttest(data)[0]} пар')

Для 11 пар


Значение уровня значимости при использовании поправки Бонферрони ($\alpha$=0.05)

In [9]:
from statsmodels.stats.multitest import multipletests

pvals = pairwise_ttest(data)[1]

p_new = multipletests(pvals, method='bonferroni')[-1]
print(f'Уровень значимости = {p_new:0.3f}')

Уровень значимости = 0.002


Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего при использовании поправки Бонферрони?

In [13]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, method='bonferroni')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 8 пар столбцов


Для какого количества пар из столбцов отвергается гипотеза о совпадении среднего при использовании метода Холма?

In [11]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, method='holm')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 8 пар столбцов


Оценка $m_0$ по методу Шведера-Спьётволла ($\lambda$ = 0.5) 

In [13]:
def shveder_spjetvoll(p_values: list, lambd_s: float = 0.5) -> float:
    len_bigger_lambd = len(list(filter(lambda x: x > lambd_s, p_values)))
    return (1 / (1 - lambd_s) * (1 + len_bigger_lambd))

In [14]:
pvals = pairwise_ttest(data)[1]

print(f'mu_0 = {shveder_spjetvoll(pvals)}')

mu_0 = 6.0
